In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("hr.db")
cur = conn.cursor()
table_query = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = table_query.fetchall()
tables

[('employee',)]

In [7]:
data_query = cur.execute("SELECT * FROM employee;")
data = data_query.fetchall()

In [8]:
datadf = pd.DataFrame(data)
datadf

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,41,Yes,Sales,1,2,Life Sciences,1,1,Female,...,Yes,3,0,8,0,1,6,4,0,5
1,1,49,No,Research & Development,8,1,Life Sciences,1,2,Male,...,No,4,1,10,3,3,10,7,1,7
2,2,37,Yes,Research & Development,2,2,Other,1,4,Male,...,Yes,3,0,7,3,3,0,0,0,0
3,3,33,No,Research & Development,3,4,Life Sciences,1,5,Female,...,Yes,3,0,8,3,3,8,7,3,0
4,4,27,No,Research & Development,2,1,Medical,1,7,Male,...,No,3,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1465,36,None,Research & Development,23,2,Medical,1,2061,Male,...,No,3,1,17,3,3,5,2,0,3
1466,1466,39,No,Research & Development,6,1,Medical,1,2062,Male,...,No,3,1,9,5,3,7,7,1,7
1467,1467,27,No,Research & Development,4,3,Life Sciences,1,2064,Male,...,Yes,4,1,6,0,3,6,2,0,3
1468,1468,49,No,Sales,2,3,Medical,1,2065,Male,...,No,3,0,17,3,2,9,6,0,8


In [12]:
query2 = cur.execute("""
                SELECT EmployeeNumber, Department, Age, Gender, Attrition
                FROM employee
""")
salesdata = query2.fetchall()

In [14]:
sales = pd.DataFrame(salesdata)
sales

,0,1,2,3,4
0,1,Sales,41,Female,Yes
1,2,Research & Development,49,Male,No
2,4,Research & Development,37,Male,Yes
3,5,Research & Development,33,Female,No
4,7,Research & Development,27,Male,No
...,...,...,...,...,...
1465,2061,Research & Development,36,Male,None
1466,2062,Research & Development,39,Male,No
1467,2064,Research & Development,27,Male,No
1468,2065,Sales,49,Male,No


In [17]:
query3 = cur.execute("""
            SELECT EmployeeNumber, EducationField, Age, Gender, Attrition
            FROM employee
            WHERE EducationField IS 'Life Sciences'
""")

fielddata = query3.fetchall()

In [19]:
field = pd.DataFrame(fielddata)
field

,0,1,2,3,4
0,1,Life Sciences,41,Female,Yes
1,2,Life Sciences,49,Male,No
2,5,Life Sciences,33,Female,No
3,8,Life Sciences,32,Male,No
4,11,Life Sciences,30,Male,No
...,...,...,...,...,...
601,2046,Life Sciences,45,Female,None
602,2048,Life Sciences,40,Male,None
603,2049,Life Sciences,35,Male,None
604,2052,Life Sciences,35,Female,None


In [25]:
cur.execute("DROP TABLE sales")

In [27]:
cur.execute("CREATE TABLE sales (EmployeeNumber, Department, Age, Gender, Attrition)")
cur.executemany("""
            INSERT INTO sales
            VALUES(?,?,?,?,?)
""",sales.values)

In [31]:
cur.execute("CREATE TABLE field (EmployeeNumber, EducationField, Age, Gender, Attrition)")
cur.executemany("""
            INSERT INTO field
            VALUES(?,?,?,?,?)
""", field.values)

In [36]:
joined = cur.execute("""
        SELECT *
        FROM field as f
        INNER JOIN sales as s
        ON f.EmployeeNumber = s.EmployeeNumber;   
""")

joineddb = joined.fetchall()

In [37]:
sandf = pd.DataFrame(joineddb)
sandf

,0,1,2,3,4,5,6,7,8,9
0,1,Life Sciences,41,Female,Yes,1,Sales,41,Female,Yes
1,2,Life Sciences,49,Male,No,2,Research & Development,49,Male,No
2,5,Life Sciences,33,Female,No,5,Research & Development,33,Female,No
3,8,Life Sciences,32,Male,No,8,Research & Development,32,Male,No
4,11,Life Sciences,30,Male,No,11,Research & Development,30,Male,No
...,...,...,...,...,...,...,...,...,...,...
601,2046,Life Sciences,45,Female,None,2046,Sales,45,Female,None
602,2048,Life Sciences,40,Male,None,2048,Research & Development,40,Male,None
603,2049,Life Sciences,35,Male,None,2049,Research & Development,35,Male,None
604,2052,Life Sciences,35,Female,None,2052,Research & Development,35,Female,None
